In [105]:
import pandas as pd
import biogeme.database as db
import biogeme.biogeme as bio

pandas = pd.read_table("tripdata.dat")
database = db.Database("tripdata",pandas)

# The Pandas data structure is available as database.data. Use all the
# Pandas functions to invesigate the database
#print(database.data.describe())

from headers import *

Below are all of the variables declarations:

In [106]:
# Alternative specific constants
ASC_AUTO_DRIVE = Beta('ASC_AUTO_DRIVE',0,None,None,0)
ASC_AUTO_PASS = Beta('ASC_AUTO_PASS',0,None,None,0)
ASC_METRO = Beta('ASC_METRO',0,None,None,0)
ASC_TRAIN = Beta('ASC_TRAIN',0,None,None,0)
ASC_WALK = Beta('ASC_WALK',0,None,None,1)

# Sociodemo variables
B_SEX = Beta('B_SEX',0,None,None,0) # 1 if female; 0 if otherwise
B_AGE = Beta('B_AGE',0,None,None,0) # age
B_EMP = Beta('B_EMP',0,None,None,0) # parttime or fulltime
B_WKWK = Beta('B_WKWK',0,None,None,0) # work week
B_INC = Beta('B_INC',0,None,None,0) # income
B_NVEH = Beta('B_NVEH',0,None,None,0) # number of household vehicles

# Generalized travel variables
B_IVTT = Beta('B_IVTT',0,None,None,0) # in vehicle travel time
B_DIST = Beta('B_DIST',0,None,None,0) # distance

# Car-related variables
B_DLIC = Beta('B_DLIC',0,None,None,0) # drivers license 1 or 0

# Train-related variables
B_NTERM = Beta('B_NTERM',0,None,None,0) # commuter train access station
B_GFARE = Beta('B_GFARE',0,None,None,0) # commuter train fare
B_GWAIT = Beta('B_GWAIT',0,None,None,0) # commuter train wait time
B_GACCA = Beta('B_GACCA',0,None,None,0) # commuter train wait time
B_EGRESS = Beta('B_GACCA',0,None,None,0) # commuter train wait time


# Metro-related variables
B_MSUB = Beta('B_MSUB',0,None,None,0) # closest metro station to destination
B_MDIS = Beta('B_MDIS',0,None,None,0) # distance from metro to destination
B_TFARE = Beta('B_TFARE',0,None,None,0) # transit fare
B_TWAIT = Beta('B_TWAIT',0,None,None,0) # transit wait time

The next cell defines new variables in order to exclude crazy values

In [107]:
# Updating variables to account for 'missing' values 
nveh_fix =  DefineVariable('nveh_fix', nveh * (nveh != 9), database)
gfare_fix = DefineVariable('gfare_fix', gfare * (gfare != 9999), database)
tfare_fix = DefineVariable('tfare_fix', tfare * (tfare != 9999), database)
gwait_fix = DefineVariable('gwait_fix', gwait * (gwait != 9999), database)
twait_fix = DefineVariable('twait_fix', twait * (twait != 9999), database)

In [108]:
V1 = ASC_AUTO_DRIVE + \
     B_DIST * trdist + \
     B_DLIC * dlic + \
     B_IVTT * ivtt1
V2 = ASC_AUTO_PASS + \
     B_NVEH * nveh_fix + \
     B_IVTT * ivtt2
V3 = ASC_METRO + \
     B_TFARE * tfare_fix + \
     B_TWAIT * twait_fix + \
     B_IVTT * ivtt3
V4 = ASC_TRAIN + \
     B_NTERM * nterm + \
     B_GFARE * gfare_fix + \
     B_GWAIT * gwait_fix + \
     B_IVTT * ivtt4
V5 = ASC_WALK + \
     B_AGE * age * age2 + \
     B_IVTT * ivtt5

In [109]:
# Associate utility functions with the numbering of alternatives
V = {1: V1,
     2: V2,
     3: V3,
     4: V4,
     5: V5}

# Associate the availability conditions with the alternatives

av = {1: avail1,
      2: avail2,
      3: avail3,
      4: avail4,
      5: avail5}

logprob = bioLogLogit(V,av,choice)
biogeme  = bio.BIOGEME(database,logprob) # This is my log likelihood function
biogeme.modelName = "MNL_Model_final"
results = biogeme.estimate()

# Print the estimated values
betas = results.getBetaValues()
for k,v in betas.items():
    print(f"{k}=\t{v:.3g}")

# Get the results in a pandas table
pandasResults = results.getEstimatedParameters()
print(pandasResults)

ASC_AUTO_DRIVE=	-75.6
ASC_AUTO_PASS=	-10.5
ASC_METRO=	-8.04
ASC_TRAIN=	-5.44
B_AGE=	1.14
B_DIST=	-0.0348
B_DLIC=	66.6
B_GFARE=	-1.33
B_GWAIT=	-0.213
B_IVTT=	-0.0325
B_NTERM=	-0.0727
B_NVEH=	-0.41
B_TFARE=	-0.849
B_TWAIT=	-0.0223
                    Value    Std err    t-test       p-value  Rob. Std err  \
ASC_AUTO_DRIVE -75.634631  44.329743 -1.706182  8.797411e-02      0.533778   
ASC_AUTO_PASS  -10.509120  88.659436 -0.118534  9.056449e-01      1.067715   
ASC_METRO       -8.042496  88.660274 -0.090711  9.277219e-01      1.118023   
ASC_TRAIN       -5.437184  88.664409 -0.061323  9.511018e-01      1.465030   
B_AGE            1.143979        NaN  0.000000  1.000000e+00           NaN   
B_DIST          -0.034805   0.011763 -2.958919  3.087204e-03      0.009657   
B_DLIC          66.601814  44.329743  1.502418  1.329891e-01      0.533778   
B_GFARE         -1.327526   0.219460 -6.049049  1.457034e-09      0.225722   
B_GWAIT         -0.212836   0.435947 -0.488216  6.253969e-01      0.4

C:\Users\rafva\Anaconda3\lib\site-packages\biogeme\results.py:168: RuntimeWarning: invalid value encountered in sqrt
  self.data.betas[i].setStdErr(np.sqrt(self.data.varCovar[i,i]))
C:\Users\rafva\Anaconda3\lib\site-packages\biogeme\results.py:182: RuntimeWarning: invalid value encountered in sqrt
  self.data.betas[i].setRobustStdErr(np.sqrt(self.data.robust_varCovar[i,i]))
